In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.utils import np_utils

import os
from numpy import argmax
from keras.preprocessing.image import load_img, img_to_array, array_to_img

Using TensorFlow backend.


In [2]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32))/255
X_test = (X_test.astype(np.float32))/255
X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

#Y_train = np_utils.to_categorical(Y_train, 10)
Y_test = np_utils.to_categorical(Y_test, 10)

We will using small learning set for "pre-train"  - 10 images of each digit. Quality of it set very important - all samples must be undoubted and covering all possible writing styles of digits. 

In [3]:
path = './samples/'
x_learn = []
y_learn = []

n = 0
while n<10:
    path = './samples/' + str(n) + '/'
    for image_path in os.listdir(path):
        img = load_img(path + image_path, color_mode = "grayscale")
        img = (img_to_array(img))/255 
        x_learn.append(img)
        y_learn.append(n)
    n+=1

x_learn = np.array(x_learn)
y_learn = np.array(y_learn)
y_learn = np_utils.to_categorical(y_learn, 10)

In [4]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss="categorical_crossentropy", optimizer='Adam', metrics=["accuracy"])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        9232      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
__________

Training model on learning batch until more than 98.5% samples will be predicted with probability >50%.

In [5]:
while True:
    model.fit(x_learn, y_learn, batch_size=10, shuffle=True, epochs=1, verbose =0)
    predicts = model.predict(X_train[:1000])
    n = 0
    for predict in predicts:
        category = argmax(predict)
        if predict[category]>0.5:
            n = n +1
    #print(str(n)+'/'+str(len(predicts)))
    if n >= 985:
        break
scores = model.evaluate(X_test, Y_test, verbose=0)
print('X_test accuracy:', scores[1]*100, '%')

X_test accuracy: 87.02 %


Then, we select all samples with predict accuracy above then threshold and continue training batch by batch, epoch by epoch. Every epoch we can reduce threshold and include in training set more uncertain images, as overall accuracy improve.

In [6]:
epochs = 5
batch_size = 10000
threshold = 0.91

for epoch in range(epochs):
    print('Epoch:', int(epoch+1), '/', epochs)
    for index in range(int(len(X_train)/batch_size)):
        print('Batch:', str(int(index+1)).zfill(2), '/', int(len(X_train)/batch_size), end='')
        
        batch = model.predict(X_train[index*batch_size:(index+1)*batch_size])
        
        x_self_train=[]
        y_self_train=[]
        for i in range(len(batch)):
            category = argmax(batch[i])
            if batch[i][category]>=threshold:
                x_self_train.append(X_train[(index*batch_size)+i])
                y_self_train.append(category)
            
        x_self_train = np.array(x_self_train)
        y_self_train = np.array(y_self_train)
        y_self_train = np_utils.to_categorical(y_self_train, 10)
        
        print(' : Accepted :', str(len(x_self_train)/batch_size*100),'% images')
        model.fit(x_self_train, y_self_train, batch_size=50, shuffle=True, epochs=5, verbose =0)
        
        scores = model.evaluate(X_test, Y_test, verbose=0)
        print('Accuracy:', scores[1]*100, '%',)
    threshold-=0.1

Epoch: 1 / 5
Batch: 01 / 6 : Accepted : 81.17999999999999 % images
Accuracy: 90.85 %
Batch: 02 / 6 : Accepted : 85.26 % images
Accuracy: 92.71 %
Batch: 03 / 6 : Accepted : 90.83 % images
Accuracy: 94.05 %
Batch: 04 / 6 : Accepted : 92.42 % images
Accuracy: 94.68 %
Batch: 05 / 6 : Accepted : 93.23 % images
Accuracy: 95.26 %
Batch: 06 / 6 : Accepted : 94.69 % images
Accuracy: 95.83 %
Epoch: 2 / 5
Batch: 01 / 6 : Accepted : 97.0 % images
Accuracy: 96.14 %
Batch: 02 / 6 : Accepted : 96.5 % images
Accuracy: 96.32 %
Batch: 03 / 6 : Accepted : 96.66 % images
Accuracy: 96.65 %
Batch: 04 / 6 : Accepted : 97.5 % images
Accuracy: 96.88 %
Batch: 05 / 6 : Accepted : 97.57000000000001 % images
Accuracy: 97.14 %
Batch: 06 / 6 : Accepted : 98.08 % images
Accuracy: 97.2 %
Epoch: 3 / 5
Batch: 01 / 6 : Accepted : 98.54 % images
Accuracy: 97.25 %
Batch: 02 / 6 : Accepted : 98.44000000000001 % images
Accuracy: 97.24 %
Batch: 03 / 6 : Accepted : 98.53 % images
Accuracy: 97.43 %
Batch: 04 / 6 : Accepted : 98